# Библиотеки

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import os
from natsort import natsorted, ns

import keras
from keras.models import Model, Sequential, model_from_json
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils

import cv2
import imutils
from tqdm import tqdm

from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


# Загрузка и обработка данных

In [2]:
def PathDetermination(user):
    if user == 'litvin_home':
        core_dir = 'C:\\Kaggle\\Understanding the Amazon from Space\\'
    
    if user == 'litvin_office':
        core_dir = 'D:\\Kaggle\\Understanding the Amazon from Space\\'
        train_dir = core_dir + 'train-jpg\\'
        test_dir = core_dir + 'test-jpg\\'
        add_test_dir = core_dir + 'test-jpg-additional\\'
    
    if user == 'savina':
        core_dir = 'Z:\\Kaggle Amazon Rainforest\\'
        train_dir = ''
        test_dir = 'C:\\Users\\horch\\Desktop\\Local_data\\test-jpg\\'
        add_test_dir = 'C:\\Users\\horch\\Desktop\\Local_data\\test-jpg-additional\\'
    
    return core_dir, train_dir, test_dir, add_test_dir

In [3]:
core_dir, train_dir, test_dir, add_test_dir = PathDetermination('litvin_office')
#core_dir, test_dir, add_test_dir = path_determination('savina')

test_data_names = natsorted(os.listdir(test_dir), key=lambda y: y.lower())
add_test_data_names = os.listdir(add_test_dir)


cathegories = ['agriculture', 'artisinal_mine', 'bare_ground', 
                      'blooming', 'blow_down', 'clear', 'cloudy', 'conventional_mine', 
                      'cultivation', 'habitation', 'haze', 'partly_cloudy', 'primary', 
                      'road', 'selective_logging', 'slash_burn', 'water']

train_data = pd.read_csv(core_dir + 'train_v2.csv')
train_data.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


# Функции

In [4]:
def BinimialPrediction(x, treshold = 0.5):
    result = np.zeros(x.shape)
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if x[i, j] >= treshold:
                result[i, j] = 1
    
    return(result)

def FBettaScore(x_true, x_predicted, betta = 2):
    if len(x_true) == len(x_predicted):
        tp = 0
        fp = 0
        fn = 0

        for i in range(len(x_predicted)):
            if x_true[i] == 1 and x_predicted[i] == 1:
                tp += 1
            
            if x_true[i] == 0 and x_predicted[i] == 1:
                fp += 1
            
            if x_true[i] == 1 and x_predicted[i] == 0:
                fn += 1
        
        if tp == 0 or (tp + fp) == 0 or (tp + fn) == 0:
            return(0)
        else:
            precision = tp/(tp + fp)
            recall = tp/(tp + fn)
            
            return((1 + betta**2)*precision*recall/(betta**2*precision + recall))
    else:
        print('FBettaScore error! len(x_true) != len(x_predicted)')

def AvgFBettaScore(x_true, x_predicted, betta = 2):
    result = 0
    n = x_true.shape[0]
    
    x_predicted = BinimialPrediction(x_predicted)
    
    for i in range(n):
        result += FBettaScore(x_true[i, :], x_predicted[i, :], betta)
    
    return(result/n)


# Вычисления

формирование обучающей выборки

In [5]:
def DataPreperation(data_type, img_size = (32, 32), rotation = [0], 
                    test_img_dir = [test_dir, add_test_dir], shuffle = False):
    if data_type == 'train':
        X = []
        Y = []
        
        for img_name in tqdm(train_data.image_name.values):
    
            img = cv2.imread(train_dir + img_name + '.jpg')
            img_resized = cv2.resize(img, img_size)
            
            img_tags = train_data[train_data['image_name'] == img_name]['tags'].values[0].split(' ')
            y = np.zeros(len(cathegories))
            
            for i in range(len(y)):
                if cathegories[i] in img_tags:
                    y[i] = 1
            
            for angle in rotation:
                img = imutils.rotate(img_resized, angle)
                X.append(img)
                Y.append(y) 
        
        X = np.array(X, np.float16) / 255.
        Y = np.array(Y)
        
        if shuffle:
            ind = np.random.choice(np.arange(X.shape[0]), X.shape[0], replace= False)
            X = X[ind]
            Y = Y[ind]
        
        return X, Y
    
    if data_type == 'test':
        X_test = []
        X_test_names = []

        for img_dir in test_img_dir:
            img_dir_names = os.listdir(img_dir)
            
            for img_name in tqdm(img_dir_names):
                if img_name.endswith("jpg"):
                    img = cv2.imread(img_dir + img_name)
                    img = cv2.resize(img, img_size)
                    
                    X_test.append(img)
                    X_test_names.append(img_name)
    
        X_test = np.array(X_test, np.float16) / 255.
        
        return X_test, X_test_names

In [6]:
X, Y = DataPreperation('train', rotation = [0, 90, 180, 270], shuffle= True)

100%|████████████████████████████████████| 40479/40479 [08:08<00:00, 82.91it/s]


разбиение выборки на обучение и тест

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Настройка и  обучение сети для всех категорий

In [8]:
batch_size = 128 # in each iteration, we consider 32 training examples at once
num_epochs = 50 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [9]:
# загрузка изображений и их приведение к подходящему для обработки виду
num_train = X.shape[0]
depth = 32 
height = 32
width = 3 

num_classes = len(cathegories)

In [10]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras!

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)

conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)

conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)

# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax

flat = Flatten()(drop_2)

hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)

out = Dense(num_classes, activation='sigmoid')(drop_3)

model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(x_train, y_train, # Train the model using the training set...
          batch_size=batch_size, nb_epoch=num_epochs,
          verbose=1, validation_data = (x_test, y_test))


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:27: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:35: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 113341 samples, validate on 48575 samples
Epoch 1/50
113341/113341 [==============================] - 644s - loss: 0.1896 - acc: 0.9259 - val_loss: 0.1530 - val_acc: 0.9400
Epoch 2/50
113341/113341 [==============================] - 642s - loss: 0.1461 - acc: 0.9427 - val_loss: 0.1323 - val_acc: 0.9473
Epoch 3/50
113341/113341 [==============================] - 641s - loss: 0.1357 - acc: 0.9464 - val_loss: 0.1243 - val_acc: 0.9501
Epoch 4/50
113341/113341 [==============================] - 642s - loss: 0.1290 - acc: 0.9492 - val_loss: 0.1234 - val_acc: 0.9515
Epoch 5/50
113341/113341 [==============================] - 642s - loss: 0.1251 - acc: 0.9509 - val_loss: 0.1179 - val_acc: 0.9535
Epoch 6/50
113341/113341 [==============================] - 640s - loss: 0.1205 - acc: 0.9528 - val_loss: 0.1176 - val_acc: 0.9539
Epoch 7/50
113341/113341 [==============================] - 640s - loss: 0.1175 - acc: 0.9541 - val_loss: 0.1136 - val_acc: 0.9553
Epoch 8/50
113341/113341 [======

In [11]:
# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("test_model_50e rotate.json", "w")
json_file.write(model_json)
json_file.close()

model.save_weights("test_model_50e rotate.h5")

## Делаем предсказание

In [12]:
# load json and create model
json_file = open('test_model_50e rotate.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights('test_model_50e rotate.h5')
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Loaded model from disk


функция для формирования тестовой выборки

In [13]:
X_test, X_test_names = DataPreperation('test')

100%|████████████████████████████████████| 20522/20522 [03:51<00:00, 88.50it/s]


In [21]:
def MakePrediction(data, model, img_names = X_test_names, treshold = 0.4, file_name = 'prediction.csv'):
    if type(treshold) == float:
        prediction = model.predict(data)
        f = open(file_name, 'w')
        f.write('image_name,tags\n')
        
        for i in tqdm(range(prediction.shape[0])):
            line = img_names[i][0:-4] + ','
            for j in range(prediction.shape[1]):
                if prediction[i, j] >= treshold:
                    line += cathegories[j] + ' '
            
            f.write(line + '\n')
        
        f.close()
    
    elif type(treshold) == list:
        if len(treshold) == data.shape[1]:
            prediction = model.predict(data)
            f = open(file_name, 'w')
            f.write('image_name,tags\n')

            for i in tqdm(range(prediction.shape[0])):
                line = img_names[i][0:-4] + ','
                for j in range(prediction.shape[1]):
                    if prediction[i, j] >= treshold[j]:
                        line += cathegories[j] + ' '
                
                f.write(line + '\n')
            
            f.close()
        
        else:
            print('wrong treshold length. must be %d' %data.shape[1])
    
    else:
        print('treshold type must be float or list')

In [22]:
MakePrediction(X_test, loaded_model, treshold = 0.2, file_name = 'prediction 02 rot.csv')

100%|█████████████████████████████████| 61191/61191 [00:02<00:00, 30143.35it/s]
